## IoT Enegry Solution Databricks Notebook App

In [4]:
import numpy as np
import pandas as pd

import warnings

import plotly.graph_objs as go
import plotly

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [0]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

!pip install https://sparkflows-release.s3.amazonaws.com/fire/jupyter-docker/firenotebookwheel/#fire_notebook-3.1.0-py3-none-any.whl
#!pip install https://sparkflows-cft-2.s3.us-east-1.amazonaws.com/test-sdk2/fire_notebook-3.1.0-py3-none-any.whl
from fire_notebook.output.workflowcontext import RestWorkflowContext

In [0]:
# Widgets used to pass value
dbutils.widgets.text("job-id", "100")
dbutils.widgets.text("postback-url", "")
dbutils.widgets.text("device_id", "meter_2")
dbutils.widgets.text("histogram_param", "voltage")
dbutils.widgets.text("linegraph_param", "energy_generation")
dbutils.widgets.text("boxplot_param", "power")

In [0]:
# Retrieve the widget values
jobId = dbutils.widgets.get("job-id")
webserverURL = dbutils.widgets.get("postback-url")
device_id = dbutils.widgets.get("device_id")
histogram_param = dbutils.widgets.get("histogram_param")
linegraph_param = dbutils.widgets.get("linegraph_param")
boxplot_param = dbutils.widgets.get("boxplot_param")

In [0]:
print(f"Device ID: {device_id}")
print(f"Histogram param: {histogram_param}")
print(f"Linegraph param: {linegraph_param}")

In [0]:
#Creating an instance 
restworkflowcontext = RestWorkflowContext(webserverURL, jobId)
message="50"
restworkflowcontext.outputProgress(id=9, title="Progress", progress=message)

In [0]:
spark = SparkSession.builder.appName("IoTEnergyConsumption").getOrCreate()

In [0]:
#/FileStore/tables/IoTEnergySolutionsEDA-2.csv
df1 = spark.read.format("csv").option("header", "true").load("/FileStore/tables/IoTEnergySolutionsEDA-2.csv")

In [0]:
restworkflowcontext.outDataFrame(id=9, title="PySpark Dataframe", df=df1, num_rows=3)

### Introduction to the IoT Energy Solution Databricks Notebook App

In [0]:
# Exploring IOT Energy Data - EDA html page
htmlstr1 = '''<div style="border-radius: 5px;">
    <div style="padding: 0.75rem 1.25rem; margin-bottom: 0; background-color: #0c9a86;
    border-bottom: 1px solid rgba(30, 30, 30, 0.125); border-top-right-radius: 5px; border-top-left-radius: 5px;
    color: #ffffff; font-family: sans-serif;">
        <h3 style="color: #fff; margin: 0;">Exploring IOT Energy Data</h3>
    </div>
    <div style="margin-top: 20px;"></div>
</div>

<div style="margin-top: 20px;">
    <h4 style="color: #3178d6;">Overview</h4>
    <ul style="margin-left: 10px; list-style-type: none; padding: 0;">
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Users can analyze IoT data by computing summary statistics to understand metrics like energy generation and consumption across devices. They can visualize these metrics using histograms to explore their distributions and frequencies. Box plots help identify outliers and compare metric distributions across different devices, aiding in anomaly detection and trend analysis.
        </li>
    </ul>
</div>
<div style="margin-top: 20px;">
    <h4 style="color: #3178d6;">Actions</h4>
    <ul style="margin-left: 10px; list-style-type: none; padding: 0;">
        <!-- Action 1 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Select a Device for which Energy Generation and Energy Consumption graph has to be plotted.
        </li>
        <!-- Action 2 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Select one or multiple Devices under the Select Device.
        </li>
        <!-- Action 3 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Click on 'Explore Data' to perform the analysis.
        </li>
    </ul>
</div>
<div style="margin-top: 20px;">
    <h4 style="color: #3178d6;">Outputs</h4>
    <ul style="margin-left: 10px; list-style-type: none; padding: 0;">
        <!-- Output 1 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Chart illustrating the Box Plot for all the devices.
        </li>
        <!-- Output 2 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Chart displaying the Correlation Table and Correlation Matrix.
        </li>
        <!-- Output 3 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            List of Devices.
        </li>
        <!-- Output 4 -->
        <li style="position: relative; margin-bottom: 10px; padding-left: 40px;">
            <span style="width: 18px; height: 18px; background-color: #0c9a86; color: #fff; border-radius: 50%; position: absolute; left: 10px; display: flex; align-items: center; justify-content: center; font-size: 20px; font-weight: 600;">+</span>
            Timeseries plot of Energy Consumption and Energy Generation for the selected devices.
        </li>
    </ul>
</div>'''



restworkflowcontext.outHTML(title="Exploring IOT Energy Data", text=htmlstr1)

### Line plot for selected metrics filtered by device ID

In [0]:
restworkflowcontext.outStr(9, "Test String", text="Input the device ID in the device_id parameter")
restworkflowcontext.outStr(9, "Test String", text="Device IDs: \n1. meter_1\n2. meter_2\n2. inverter_1\n2. inverter_2\n2. feeder_1\n2. feeder_2")

In [0]:
restworkflowcontext.outStr(9, "Test String", text="Input the parameter to plot line graph in the linegraph_param")
restworkflowcontext.outStr(9, "Test String", text="Options: \n1. energy_consumption\n2. energy_genertaion")

In [0]:
device_id = dbutils.widgets.get("device_id")
print(f"Device ID: {device_id}")
linegraph_param = dbutils.widgets.get("linegraph_param")
print(f"Linegraph param: {linegraph_param}")

In [0]:
pandas_df = df1.toPandas()
df_filtered_pd = pandas_df[pandas_df['device_id']==device_id]
df_filtered_pd.head()

In [0]:
import plotly.express as px

fig = px.line(df_filtered_pd, x='timestamp', y=linegraph_param, title=f"{linegraph_param} for {device_id}")
test = plotly.offline.plot(fig, output_type='div')
energy_consumption = f'{test}'
restworkflowcontext.outPlotly(title=f"Energy consumption for {device_id}", text=energy_consumption)

### Histogram for the selected column

In [0]:
restworkflowcontext.outStr(9, "Test String", text="Input the histogram_param to plot the histogram")
restworkflowcontext.outStr(9, "Test String", text="Options: \n1. current\n2. voltage\n2. power\n2. frequency\n2. energy_genertaion\n2. energy_consumption")

In [0]:
histogram_param = dbutils.widgets.get("histogram_param")
print(f"Histogram param: {histogram_param}")

In [0]:
# Create histogram
fig = px.histogram(df_filtered_pd, x=histogram_param, title=f'Histogram for {histogram_param}')
test = plotly.offline.plot(fig, output_type='div')
histogram = f'{test}'
restworkflowcontext.outPlotly(title=f"Histogram of {histogram_param}", text=histogram)

### Boxplot for selected column

In [0]:
restworkflowcontext.outStr(9, "Test String", text="Input the boxplot_param to plot the boxplot")
restworkflowcontext.outStr(9, "Test String", text="Options: \n1. current\n2. voltage\n2. power\n2. frequency\n2. energy_genertaion\n2. energy_consumption")

In [0]:
boxplot_param = dbutils.widgets.get("boxplot_param")
print(f"Boxplot param: {boxplot_param}")

In [0]:
# Create box plot
fig = px.box(pandas_df, y=boxplot_param, title=f'Box Plot for {boxplot_param}')
test = plotly.offline.plot(fig, output_type='div')
corr_matrix = f'{test}'
restworkflowcontext.outPlotly( title=f"Boxplot for {boxplot_param}", text=corr_matrix)